### Introduction

The DreaMS project introduces a new data format for MS/MS data. This format is based on HDF5 (Hierarchical Data Format 5) and is particularly suitable for machine learning applications. HDF5 is a binary, non-human-readable format that stores MS/MS data as a collection of tensors. One of these tensors contains MS/MS spectra, while the others contain metadata such as precursor m/z values. There are three main advantages of this data format compared to existing ones such as `.mzML` or `.mgf`. 
1. It allows for handling huge files containing millions of mass spectra by leveraging the on-disk nature of HDF5. This means that data can be read or processed efficiently in chunks or spectrum-by-spectrum without loading the entire dataset into memory. 
2. The tensor-based nature of HDF5 allows for seamless integration of MS/MS spectra into machine learning frameworks such as scikit-learn or PyTorch. 
3. The HDF5 format supports advanced data compression techniques, significantly reducing file size and storage requirements.

Notice that the .hdf5 data format can also be used to store complete LC-MS/MS runs, including MS1 data. We omit it in this tutorial for simplicity.

### Opening files and converting them to .hdf5

The DreaMS Python package implements the conversion of common mass spectrometry data formats, such as `.mzML` or `.mgf`, to `.hdf5`. The class responsible for this is named `MSData`. Let's look at an example of opening an `.mzML` file.

In [5]:
from dreams.utils.data import MSData
msdata = MSData.from_mzml('../data/MSV00008490/G73954_1x_BC8_01_17287.mzML')

Loading dataset G73954_1x_BC8_01_17287 into memory (1930 spectra)...


Once the `.mzML` file is parsed with `MSData.from_mzml`, it is auto converted to an `.hdf5` format and stored on disk.

In [10]:
!ls -lhtr ../data/MSV00008490/ | grep G73954_1x_BC8_01_17287

c-rw-r--r--@ 1 roman  staff    12M Jul 16 23:29 G73954_1x_BC8_01_17287.mzML
-rw-r--r--@ 1 roman  staff   4.0M Jul 17 21:32 G73954_1x_BC8_01_17287.hdf5


Similarly, one can open and convert an `.mgf` file. If a file does not follow a standard specification for metadata names, such as naming a precursor m/z as `PEPMASS`, one can specify the correct names using constructor arguments such as `prec_mz_col`.

In [35]:
msdata = MSData.from_mgf('../data/MassSpecGym.mgf', prec_mz_col='PRECURSOR_MZ')

We can see that the data folder now contains a new `.hdf5` file with the same base name. The new file is approximately three times smaller in size, which is a result of two factors: `.hdf5` retains only MS/MS spectra and stores them in a memory-efficient tensor-based format.

The following sections demonstrate how the content of the `.hdf5` file can be manipulated using the methods of the `MSData` class. Alternatively, one can handle the `.hdf5` file manually using the [h5py library](https://docs.h5py.org/en/stable/index.html).

### Handling MS/MS spectra

In [12]:
spectra = msdata.get_spectra()
print(f'"spectra" object is a {type(spectra)} tensor of shape {spectra.shape} containing {spectra.dtype} elements.')

"spectra" object is a <class 'numpy.ndarray'> tensor of shape (1930, 2, 128) containing float64 elements.


The dimensions of the tensor have the following interpretation. First dimension (1930 elements) is a number of spectra. Second dimension (2 elements) represents m/z and intensity values, where first element is m/z values and second element are intensity values. Third dimension (128 elements) represents the number of signals in each spectrum. One can notice that since a tensor-shaped structre demans each dimension to have a fixed number of elements, each spectrum has the same number of signals. This was achieved during the conversion of .mzML to .hdf5. Let us look, for example, at the first spectrum.

In [15]:
spectra[0].T

array([[337.10501099, 720.        ],
       [338.10757446, 138.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [  0.        ,   0.        ],
 

The spectrum has only two signals, with the rest of the elements being zeros. These zeros, often referred to as padding zeros, are used to fill the tensor up to a desired shape (let's denote this as n, where n = 128 in our case). Conversely, when a spectrum has more than n signals, only the n most intense signals are selected and stored in the tensor. The remaining signals are discarded and not stored in the file. The value of n can be controlled using the `n_highest_peaks` argument of the `MSData` constructor. To ensure that no signals are lost, one can set `n_highest_peaks` to the maximum number of peaks in the entire dataset. Since the padding zeros are effectively compressed in HDF5, this will not result in excessively large file sizes.

To remove the padding zeros, one can use the `unpad_peak_list` function from the DreaMS package.

In [26]:
from dreams.utils.spectra import unpad_peak_list
unpad_peak_list(spectra[0]).T

array([[337.10501099, 720.        ],
       [338.10757446, 138.        ]])

### Handling metadata

To retrieve all columns parsed from the input file, one can use the `columns` method.

In [36]:
msdata.columns()

['ADDUCT',
 'COLLISION_ENERGY',
 'FOLD',
 'FORMULA',
 'INCHIKEY',
 'INSTRUMENT_TYPE',
 'PARENT_MASS',
 'PRECURSOR_FORMULA',
 'SIMULATION_CHALLENGE',
 'SMILES',
 'precursor_mz',
 'spectrum']

To get elements of a column as a complete column one can either user square brackers or `get_values` method. The second option is preferred, as explained in the "Working with huge files" chapter of this tutorial.

In [38]:
msdata.get_values('FORMULA')  # Or msdata['FORMULA']

array(['C16H17NO4', 'C16H17NO4', 'C16H17NO4', ..., 'C21H43N5O7',
       'C21H43N5O7', 'C21H43N5O7'], dtype='<U17')

In [40]:
msdata.get_values('FORMULA', 0)   # Or msdata['FORMULA'][0]

'C16H17NO4'

In [41]:
msdata.get_values('FORMULA', [0, 1, 2])  # Or msdata['FORMULA'][0, 1, 2]

array(['C16H17NO4', 'C16H17NO4', 'C16H17NO4'], dtype='<U9')

The `MSData` class provides methods for more advanced functionality, such as converting spectra and their metadata to [`matchms`](https://github.com/matchms/matchms) or merging multiple `.hdf5` files. Please refer to the class reference documentation for details.

### TODO: Working with huge files